<a href="https://colab.research.google.com/github/Josh-Been/Mining-Reddit-Instagram/blob/master/Mine_Instagram_Images_by_Hashtag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Baylor Libraries: Digital Scholarship](https://cpb-us-w2.wpmucdn.com/blogs.baylor.edu/dist/7/7192/files/2019/08/cropped-DigitalScholarshipblog_header-2019-08-30-1.jpg)

# Mine Instagram by Hashtag

Primary Libraries Used

*   OCR Text from Images [Github: tesseract-ocr](https://github.com/tesseract-ocr/tesseract)
*   Recognize Objects in Images [Github: ImageAI](https://github.com/OlafenwaMoses/ImageAI)

Thank you to the following helpful guides

*   [How to Scrape Data from Instagram using Python](https://www.promptcloud.com/blog/how-to-scrape-instagram-data-using-python/)
*   [Bhadresh Savani: OCR from Image using Pytesseract in Python on Colab Notebook?](https://medium.com/@bhadreshpsavani/how-to-use-tesseract-library-for-ocr-in-google-colab-notebook-5da5470e4fe0)
*   [Object Detection with 10 lines of code](https://towardsdatascience.com/object-detection-with-10-lines-of-code-d6cb4d86f606)

02/2020, Josh Been

# Step 1 - Specify Hashtags to Search

Enter the hashtags you want to mine images. Separate hashtags by a comma.

For example, hashes = ['baylor', 'waco']

Run the following snippet. Wait until it specifies complete before moving to the next snippet.

In [0]:
hashes = ['']
cursor='  >> '
print(cursor, 'Move onto the next snippet.')

# Step 2 - Write List of Images to File

Run the following snippet.

When it is complete, check Files on the left for a text file containing image urls of each hashtag included in step #1.

In [0]:
import pandas, requests, urllib.request, urllib.parse, urllib.error, ssl, json
from bs4 import BeautifulSoup
from google.colab import output
import urllib.request

class Insta_Image_Links_Scraper:

    def getlinks(self, hashtag, url):

        html = urllib.request.urlopen(url, context=self.ctx).read()
        soup = BeautifulSoup(html, 'html.parser')
        script = soup.find('script', text=lambda t: \
                           t.startswith('window._sharedData'))
        page_json = script.text.split(' = ', 1)[1].rstrip(';')
        data = json.loads(page_json)
        print (cursor, 'Scraping links with #' + hashtag+"...........")
        for post in data['entry_data']['TagPage'][0]['graphql'
                ]['hashtag']['edge_hashtag_to_media']['edges']:
            image_src = post['node']['thumbnail_resources'][1]['src']
            hs = open(hashtag + '.txt', 'a')
            hs.write(image_src + '\n')
            hs.close()

    def main(self):
        self.ctx = ssl.create_default_context()
        self.ctx.check_hostname = False
        self.ctx.verify_mode = ssl.CERT_NONE
       
        self.content=hashes
        for hashtag in self.content:
            self.getlinks(hashtag,
                          'https://www.instagram.com/explore/tags/'
                          + hashtag + '/')


if __name__ == '__main__':
    obj = Insta_Image_Links_Scraper()
    obj.main()

print(cursor, 'Complete.')    

# Step 3 - Download Images to Folder and Zip

This step will download all of the images identified in the previous step and create a .zip file that can be downloaded to your computer.

In [0]:
import os, shutil
from google.colab import files

for hashtag in hashes:
  if not os.path.isdir(hashtag):
    os.mkdir(hashtag)
  with open(hashtag+'.txt') as f:
    content = f.readlines()
    content = [x.strip() for x in content]
  i=0
  existing_files=[]
  for link in content:
    i+=1
    # print(link)
    filename = link.split('/')[-1].split('?')[0]
    while filename in existing_files:
        filename=filename+'1'
        print('yep')
    existing_files.append(filename)
    urllib.request.urlretrieve(link, hashtag+'/'+filename)
    
  shutil.make_archive(hashtag, 'zip', hashtag)

print(cursor, 'Complete.')    

# Step 4 - OCR Text From Images

Extract text from images. Clean text and then write to new CSV spreadsheet.

In [0]:
print(cursor, 'Install Tesseract OCR')
!sudo apt install tesseract-ocr
!pip install pytesseract
output.clear()

print(cursor, 'Install Tesseract OCR')
print(cursor, 'Installation completed')
import pytesseract, shutil, os, random, glob, spacy
import pandas as pd
try:
    from PIL import Image
except ImportError:
    import Image
nlp = spacy.load("en_core_web_sm")
imgtxt=[]
urls=[]
htag=[]
for hashtag in hashes:
    print(cursor, 'Processing', hashtag)
    fileset = [file for file in glob.glob(hashtag + "/*", recursive=True)]
    i=1
    for file in fileset:
        htag.append(hashtag)
        word_list=[]
        obtained_txt=pytesseract.image_to_string(Image.open(file))
        obtained_text=obtained_txt.lower()
        doc=nlp(obtained_txt)
        for token in doc:
            word_list.append(token.text.lower().replace('\n',''))
        extractedInformation=' '.join(word_list)
        imgtxt.append(extractedInformation)
        urls.append(file.split('/')[1])
        i+=1

df_img = pd.DataFrame(htag, columns =['hashtag'])
df_img['img2txt']=imgtxt
df_img['img']=urls

df_img.to_csv('INSTAGRAM.csv', index=False)

# Step 5 - Install ImageAI

In [0]:
print(cursor,'Installing imageai')
!pip install imageai --upgrade
output.clear()
print(cursor,'Installing imageai')
import urllib.request
urllib.request.urlretrieve ("https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/resnet50_coco_best_v2.0.1.h5", "resnet50_coco_best_v2.0.1.h5")
print(cursor, 'Completed')

# Step 6 - Recognize Objects
[AI Details](https://github.com/OlafenwaMoses/ImageAI/blob/master/imageai/Detection/README.md)

In [0]:
%tensorflow_version 1.x
from imageai.Detection import ObjectDetection
import os
from IPython.display import Image

recognized_items={}
for hashtag in hashes:
    print(cursor, 'Processing hasahtag:', hashtag)
    if not os.path.isfile(hashtag+'/resnet50_coco_best_v2.0.1.h5'): 
        shutil.move('resnet50_coco_best_v2.0.1.h5', hashtag+'/resnet50_coco_best_v2.0.1.h5')
    execution_path = hashtag
    fileset = [file for file in glob.glob(hashtag + "/*", recursive=True)]
    i=0
    
    # [:10] is first 10 only
    for img in fileset[:10]:
        i+=1 
        img_=img.split('/')[1]
        print(cursor, 'processing image', i)
        detector = ObjectDetection()
        detector.setModelTypeAsRetinaNet()
        detector.setModelPath( os.path.join(execution_path , "resnet50_coco_best_v2.0.1.h5"))
        detector.loadModel()
        detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , img_), output_image_path=os.path.join(execution_path , img_.replace('.jpg','_new.jpg')))
        if i==1:
            output.clear()
            print(cursor, 'Processing hasahtag:', hashtag)
            print(cursor, 'processing image', i)
        if len(detections)>0:
            display(Image(hashtag+'/'+img_.replace('.jpg','_new.jpg')))
            for eachObject in detections:
                print(eachObject["name"] , " : " , eachObject["percentage_probability"] )
                if eachObject["name"] in recognized_items:
                    recognized_items[eachObject["name"]]+=1
                else:
                    recognized_items[eachObject["name"]]=1
        else:
            print(cursor, 'No objects recognized')
    shutil.move(hashtag+'/resnet50_coco_best_v2.0.1.h5', 'resnet50_coco_best_v2.0.1.h5')

# Step 7 - Column Chart of Recognized Images

In [0]:
import pylab as pl
import numpy as np

X = np.arange(len(recognized_items))
pl.bar(X, recognized_items.values(), align='center', width=0.5)
pl.xticks(X, recognized_items.keys())
ymax = max(recognized_items.values()) + 1
pl.ylim(0, ymax)
pl.show()